# "Neural Turing Machines" paper implementation - https://arxiv.org/pdf/1410.5401.pdf

In [5]:
import random
import torch
from torch import nn
from torch import optim
import numpy as np
from model.model import EncapsulatedNTM

class CopyTaskParams:
    def __init__(self, name="copy-task", controller_size=100, controller_layers=1, num_heads=1,
                 sequence_width=8, sequence_min_len=1, sequence_max_len=20, memory_n=128, memory_m=20,
                 num_batches=50000, batch_size=1, rmsprop_lr=1e-4, rmsprop_momentum=0.9, rmsprop_alpha=0.95):
        self.name = name
        self.controller_size = int(controller_size)
        self.controller_layers = int(controller_layers)
        self.num_heads = int(num_heads)
        self.sequence_width = int(sequence_width)
        self.sequence_min_len = int(sequence_min_len)
        self.sequence_max_len = int(sequence_max_len)
        self.memory_n = int(memory_n)
        self.memory_m = int(memory_m)
        self.num_batches = int(num_batches)
        self.batch_size = int(batch_size)
        self.rmsprop_lr = float(rmsprop_lr)
        self.rmsprop_momentum = float(rmsprop_momentum)
        self.rmsprop_alpha = float(rmsprop_alpha)

class CopyTaskModelTraining:
    def __init__(self, params=None):
        if params is None:
            params = CopyTaskParams()
        self.params = params
        self.net = self.default_net()
        self.dataloader = self.default_dataloader()
        self.criterion = self.default_criterion()
        self.optimizer = self.default_optimizer()

    def default_net(self):
        net = EncapsulatedNTM(self.params.sequence_width + 1, self.params.sequence_width,
                              self.params.controller_size, self.params.controller_layers,
                              self.params.num_heads,
                              self.params.memory_n, self.params.memory_m)
        return net

    def default_dataloader(self):
        return dataloader(self.params.num_batches, self.params.batch_size,
                          self.params.sequence_width,
                          self.params.sequence_min_len, self.params.sequence_max_len)

    def default_criterion(self):
        return nn.BCELoss()

    def default_optimizer(self):
        return optim.RMSprop(self.net.parameters(),
                             momentum=self.params.rmsprop_momentum,
                             alpha=self.params.rmsprop_alpha,
                             lr=self.params.rmsprop_lr)

def dataloader(num_batches, batch_size, seq_width, min_len, max_len):
    for batch_num in range(num_batches):
        seq_len = random.randint(min_len, max_len)
        seq = np.random.binomial(1, 0.5, (seq_len, batch_size, seq_width))
        seq = torch.from_numpy(seq)

        inp = torch.zeros(seq_len + 1, batch_size, seq_width + 1)
        inp[:seq_len, :, :seq_width] = seq
        inp[seq_len, :, seq_width] = 1.0
        outp = seq.clone()

        yield batch_num + 1, inp.float(), outp.float()

In [6]:
import argparse
import json
import logging
import time
import random
import re
import sys

import attr
import torch
import numpy as np


LOGGER = logging.getLogger(__name__)

TASKS = {
    'copy': (CopyTaskModelTraining, CopyTaskParams),
}

RANDOM_SEED = 1000
REPORT_INTERVAL = 200
CHECKPOINT_INTERVAL = 1000


def get_ms():
    return time.time() * 1000


def init_seed(seed=None):
    if seed is None:
        seed = int(get_ms() // 1000)

    LOGGER.info("Using seed=%d", seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    random.seed(seed)


def progress_clean():
    print("\r{}".format(" " * 80), end='\r')


def progress_bar(batch_num, report_interval, last_loss):
    progress = (((batch_num-1) % report_interval) + 1) / report_interval
    fill = int(progress * 40)
    print("\r[{}{}]: {} (Loss: {:.4f})".format(
        "=" * fill, " " * (40 - fill), batch_num, last_loss), end='')


def save_checkpoint(net, name, args, batch_num, losses, costs, seq_lengths):
    progress_clean()

    basename = "{}/{}-{}-batch-{}".format(args.checkpoint_path, name, args.seed, batch_num)
    model_fname = basename + ".model"
    LOGGER.info("Saving model checkpoint to: '%s'", model_fname)
    torch.save(net.state_dict(), model_fname)

    train_fname = basename + ".json"
    LOGGER.info("Saving model training history to '%s'", train_fname)
    content = {
        "loss": losses,
        "cost": costs,
        "seq_lengths": seq_lengths
    }
    open(train_fname, 'wt').write(json.dumps(content))


def clip_grads(net):
    parameters = list(filter(lambda p: p.grad is not None, net.parameters()))
    for p in parameters:
        p.grad.data.clamp_(-10, 10)

In [ ]:
def train_batch(net, criterion, optimizer, X, Y):
    optimizer.zero_grad()
    inp_seq_len = X.size(0)
    outp_seq_len, batch_size, _ = Y.size()

    net.init_sequence(batch_size)

    for i in range(inp_seq_len):
        net(X[i])

    y_out = torch.zeros(Y.size())
    for i in range(outp_seq_len):
        y_out[i], _ = net()

    loss = criterion(y_out, Y)
    loss.backward()
    clip_grads(net)
    optimizer.step()

    y_out_binarized = y_out.clone().data
    y_out_binarized.apply_(lambda x: 0 if x < 0.5 else 1)

    cost = torch.sum(torch.abs(y_out_binarized - Y.data))

    return loss.item(), cost.item() / batch_size